# Pools

## Setup

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_token_columns
from mezo.datetime_utils import format_datetimes
from mezo.currency_config import POOL_TOKEN0_MAP
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [55]:
POOLS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_POOL_FEES = """
query poolFees($skip: Int!) {
fees(
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    amount0
    amount1
    contractId_
    transactionHash_
    block_number
    }
}
"""

## Get pool volume data from tigris-pools-mezo

In [56]:
TIGRIS_POOLS_SUBRAPH = 'https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/tigris-pools-mezo/1.0.0/gn'

GET_POOL_VOLUME = """
query getPoolVolumes($skip: Int!) {
  poolVolumes(
    interval: day
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
  ) {
  	timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalVolume0
    totalVolume1
  }
}
"""

In [57]:
# Instantiate subgraph
pool_volumes = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_volume_data =  pool_volumes.fetch_subgraph_data(GET_POOL_VOLUME, 'poolVolumes')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [58]:
def flatten_json_column(df, json_col, prefix=None):
    """
    Flatten any JSON column into separate columns using pd.json_normalize
    
    Args:
        df: DataFrame with column containing JSON data
        json_col: Name of the column containing JSON data
        prefix: Optional prefix for new column names (defaults to json_col + '_')
    
    Returns:
        DataFrame with flattened JSON data as new columns
    """
    if json_col not in df.columns:
        raise ValueError(f"Column '{json_col}' not found in DataFrame")
    
    # Set default prefix
    if prefix is None:
        prefix = f"{json_col}_"
    
    # Normalize the JSON data
    json_normalized = pd.json_normalize(df[json_col])
    
    # Create the result DataFrame starting with original data
    result_df = df.copy()
    
    # Add all flattened columns with prefix
    for col in json_normalized.columns:
        # Replace dots with underscores for cleaner column names
        clean_col_name = col.replace('.', '_')
        new_col_name = f"{prefix}{clean_col_name}"
        result_df[new_col_name] = json_normalized[col]
    
    return result_df

In [59]:
# Save pool volume data df

volume_df = pd.DataFrame(pool_volume_data)
display(volume_df)

# flatten col
volume_df_flattened = flatten_json_column(volume_df, 'pool')
volume_df_flattened = volume_df_flattened.drop(columns=['pool'])

,timestamp,pool,totalVolume0,totalVolume1
0,1757894400000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",35443948974042968,2740769004999900000000
1,1757894400000000,"{'name': 'Stable AMM - mUSDC/MUSD', 'token0': ...",375176260,190471736000000000000
2,1757894400000000,"{'name': 'Stable AMM - MUSD/mUSDT', 'token0': ...",45001200000000000000,0
3,1757894400000000,"{'name': 'Stable AMM - mUSDC/mUSDT', 'token0':...",0,45068043
4,1757808000000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",119064124556640625,12143684580415643558984
...,...,...,...,...
238,1751932800000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",250000000000000,0
239,1751846400000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",0,18910000000000000000
240,1751846400000000,"{'name': 'Stable AMM - mUSDC/MUSD', 'token0': ...",2010400,0
241,1750896000000000,"{'name': 'Stable AMM - MUSD/mUSDT', 'token0': ...",0,1000000


In [65]:
volume_df_flattened['pool'].unique()

# upload to bigquery

array(['btc_musd_pool', 'musdc_musd_pool', 'musd_musdt_pool',
       'musdc_musdt_pool', nan], dtype=object)

In [61]:
volume_df_flattened['pool_token0_symbol'] = volume_df_flattened['pool_token0_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

volume_df_flattened['pool_token1_symbol'] = volume_df_flattened['pool_token1_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

In [62]:
# ADD TO CURRENCY_CONFIG

TIGRIS_MAP = {
    'Volatile AMM - BTC/MUSD': 'btc_musd_pool',
    'Stable AMM - mUSDC/MUSD': 'musdc_musd_pool',
    'Stable AMM - MUSD/mUSDT': 'musd_musdt_pool',
    'Stable AMM - mUSDC/mUSDT': 'musdc_musdt_pool'
}

In [63]:
volume_df_flattened['pool'] = volume_df_flattened['pool_name'].map(TIGRIS_MAP)

In [64]:
vol_df_test = format_datetimes(volume_df_flattened, ['timestamp'])
vol_df_test.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
0,2025-09-15,35443948974042968,2740769004999900000000,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
1,2025-09-15,375176260,190471736000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
2,2025-09-15,45001200000000000000,0,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool
3,2025-09-15,0,45068043,Stable AMM - mUSDC/mUSDT,USDC,USDT,musdc_musdt_pool
4,2025-09-14,119064124556640625,12143684580415643558984,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool


In [66]:
btc_musd_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Volatile AMM - BTC/MUSD']
musd_musdt_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Stable AMM - MUSD/mUSDT']
musdc_musd_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Stable AMM - mUSDC/MUSD']

In [67]:
musdc_musd_pool

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
1,2025-09-15,375176260,190471736000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
5,2025-09-14,139549713,243500000000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
7,2025-09-13,334361270,1645524772369767321502,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
10,2025-09-12,353631316,567353540000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
14,2025-09-11,191621376,556346000000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
...,...,...,...,...,...,...,...
224,2025-07-17,4969702538,3796893018000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
225,2025-07-16,6221213169,6036966709103515625000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
232,2025-07-11,7003515,2003906250000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
237,2025-07-09,0,2009900000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool


In [68]:
musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0'].astype('float')
musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1'].astype('float')

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_65102/2472559031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0'].astype('float')
/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_65102/2472559031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1'].astype('float')


In [69]:
musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1']/1e18

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_65102/3222483767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1']/1e18


In [70]:
musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0']/1e6

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_65102/692796272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0']/1e6


In [ ]:
musdc_musd_pool['totalVolume0'].sum()

In [71]:
vol_df_test_2 = format_token_columns(vol_df_test, ['totalVolume0'], 'pool_token0_symbol')
vol_df_test_2.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
0,2025-09-15,0.035444,2740769004999900000000,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
1,2025-09-15,375.176260,190471736000000000000,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
2,2025-09-15,45.001200,0,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool
3,2025-09-15,0.000000,45068043,Stable AMM - mUSDC/mUSDT,USDC,USDT,musdc_musdt_pool
4,2025-09-14,0.119064,12143684580415643558984,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool


In [72]:
vol_df_test_3 = format_token_columns(vol_df_test_2, ['totalVolume1'], 'pool_token1_symbol')

In [29]:
from mezo.currency_utils import get_token_prices
from mezo.currency_config import TOKENS_ID_MAP

def add_usd_conversions(df, token_column, tokens_id_map, amount_columns=None):
    """
    Add USD price conversions to any token data
    
    Args:
        df: DataFrame containing token data
        token_column: Name of column containing token identifiers
        tokens_id_map: Dictionary mapping tokens to CoinGecko IDs
        amount_columns: List of amount columns to convert, or None for auto-detection
    
    Returns:
        DataFrame with USD conversion columns added
    """
    if token_column not in df.columns:
        raise ValueError(f"Column '{token_column}' not found in DataFrame")
    
    # Get token prices
    prices = get_token_prices()
    if prices is None or prices.empty:
        raise ValueError("No token prices received from API")
    
    token_usd_prices = prices.T.reset_index()
    df_result = df.copy()
    df_result['index'] = df_result[token_column].map(tokens_id_map)
    
    df_with_usd = pd.merge(df_result, token_usd_prices, how='left', on='index')
    
    # Set MUSD price to 1.0 (1:1 with USD)
    df_with_usd.loc[df_with_usd[token_column] == 'MUSD', 'usd'] = 1.0
    
    # Auto-detect amount columns if not provided
    if amount_columns is None:
        amount_columns = [col for col in df.columns if 'amount' in col.lower() and col != 'amount_usd']
    
    # Add USD conversion for each amount column
    for col in amount_columns:
        if col in df_with_usd.columns:
            usd_col_name = f"{col}_usd" if not col.endswith('_usd') else col
            df_with_usd[usd_col_name] = df_with_usd[col] * df_with_usd['usd']
    
    return df_with_usd

In [ ]:
vol_df_test_4 = add_usd_conversions(vol_df_test_3, token_column='pool_token0_symbol', tokens_id_map=TOKENS_ID_MAP, amount_columns=['totalVolume0'])
vol_df_test_4.head()

In [ ]:
vol_df_test_4['count'] = 1

In [ ]:
vol_df_test_5 = vol_df_test_4[['timestamp', 'totalVolume0', 'totalVolume1', 'pool_name',
       'pool_token0_symbol', 'pool_token1_symbol', 'pool', 'totalVolume0_usd', 'count']]

In [ ]:
vol_df_test_final = add_usd_conversions(vol_df_test_5, token_column='pool_token1_symbol', tokens_id_map=TOKENS_ID_MAP, amount_columns=['totalVolume1'])
vol_df_test_final.head()

In [ ]:
vol_df_test_final.columns

In [ ]:
volume_final = vol_df_test_final[[
    'timestamp', 
    'totalVolume0', 'totalVolume1', 
    'pool_name', 'pool',
    'totalVolume0_usd', 'totalVolume1_usd', 'count']]

In [ ]:
volume_final

In [ ]:
volume_by_pool = volume_final.groupby(['pool_name']).agg(
        total_volume_0 = ('totalVolume0_usd', 'sum'),
        total_volume_1 = ('totalVolume1_usd', 'sum')
    ).reset_index()

volume_by_pool

In [ ]:
volume_by_pool['total_volume'] = volume_by_pool['total_volume_0'] + volume_by_pool['total_volume_1']

In [ ]:
volume_by_pool

## Get pool deposits and withdrawals

### Deposits

In [ ]:
# Fetch pool fees data
pool_fees = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_fees_data = pool_fees.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

# Save pool fees data df
fees_df = pd.DataFrame(pool_fees_data)
display(fees_df)

# Flatten JSON column
fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

# Format timestamps
fees_df_formatted = format_datetimes(fees_df_flat, ['timestamp'])

# Format token columns for totalFees0 and totalFees1
fees_df_token0 = format_token_columns(fees_df_formatted, ['totalFees0'], 'pool_token0_symbol')
fees_df_token1 = format_token_columns(fees_df_token0, ['totalFees1'], 'pool_token1_symbol')

# Add USD conversions for totalFees0
fees_df_usd0 = add_usd_conversions(fees_df_token1, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])

# Add USD conversions for totalFees1 
fees_df_usd1 = add_usd_conversions(fees_df_usd0, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

# Add count column
fees_df_usd1['count'] = 1

print("Processed fees data:")
display(fees_df_usd1.head())
print(f"\nTotal fee records: {len(fees_df_usd1)}")
print(f"Date range: {fees_df_usd1['timestamp'].min()} to {fees_df_usd1['timestamp'].max()}")

fees_df_final = fees_df_usd1

In [ ]:
GET_DEPOSITS = """
    query getMints ($skip: Int!) {
    mints(
        first: 1000
        orderBy: timestamp_
        orderDirection: desc
        skip: $skip
    ) {
        timestamp_
        sender
        amount0
        amount1
        contractId_
        transactionHash_
    }
}
"""

In [ ]:
# Instantiate subgraph
pool_deposits = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_deposit_data =  pool_deposits.fetch_subgraph_data(GET_DEPOSITS, 'mints')

In [ ]:
raw = pd.DataFrame(pool_deposit_data)
display(raw)

ddf = raw.copy()

In [ ]:
from mezo.currency_config import POOL_TOKEN_PAIRS, TOKENS_ID_MAP, POOLS_MAP
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions

ddf['pool'] = ddf['contractId_'].map(POOLS_MAP)
ddf = format_datetimes(ddf, ['timestamp_'])
ddf = format_pool_token_columns(ddf, 'contractId_', POOL_TOKEN_PAIRS)
ddf = add_pool_usd_conversions(ddf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

ddf

In [ ]:
ddf.loc[ddf['pool'] == 'musdc_musd_pool']

### Withdrawals

In [ ]:
GET_WITHDRAWALS = """
query getWithdrawals($skip: Int!) {
  burns(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    to
    amount0
    amount1
    contractId_
    transactionHash_
  }
}
"""

In [ ]:
# Instantiate subgraph
pool_withdrawals = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_wit_data =  pool_withdrawals.fetch_subgraph_data(GET_WITHDRAWALS, 'burns')

In [ ]:
# Process withdrawals data in the same way as deposits
withdrawals_df = pd.DataFrame(pool_wit_data)
wdf = withdrawals_df.copy()

# Apply same processing steps as deposits
wdf['pool'] = wdf['contractId_'].map(POOLS_MAP)
wdf = format_datetimes(wdf, ['timestamp_'])
wdf = format_pool_token_columns(wdf, 'contractId_', POOL_TOKEN_PAIRS)
wdf = add_pool_usd_conversions(wdf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

print("Processed withdrawals data:")
display(wdf.head())
print(f"\nTotal withdrawals: {len(wdf)}")
print(f"Date range: {wdf['timestamp_'].min()} to {wdf['timestamp_'].max()}")

wdf

In [ ]:
wdf.loc[wdf['pool'] == 'musdc_musd_pool'].count() + 322

## Get pool fees

In [34]:
GET_TOTAL_POOL_FEES = """
query getFeeStats($skip: Int!) {
  feesStats_collection(
    interval: day
    first: 1000
    orderBy: timestamp
    orderDirection: desc
    skip: $skip
  ) {
    timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalFees0
    totalFees1
    id
  }
}
"""

In [35]:
# Instantiate subgraph
fees_raw = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
fees_data =  fees_raw.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [46]:
fees_df = pd.DataFrame(fees_data)

fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

fees_df_flat.head()

,timestamp,totalFees0,totalFees1,id,pool_name,pool_token0_symbol,pool_token1_symbol
0,1757203200000000,117708003515624,14655119259000000000,12942579003817984,Volatile AMM - BTC/MUSD,BTC,MUSD
1,1757203200000000,3782,140000000000,12933920349749248,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
2,1757203200000000,201540000000000,12776,12931300419698688,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
3,1757116800000000,271164124218750,25419311557189686445,12845688836587520,Volatile AMM - BTC/MUSD,BTC,MUSD
4,1757116800000000,0,8212,12824282719584256,Stable AMM - MUSD/mUSDT,MUSD,mUSDT


In [47]:
# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

# Format timestamps
fees_df_flat = format_datetimes(fees_df_flat, ['timestamp'])

# Format token columns for totalFees0 and totalFees1
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees0'], 'pool_token0_symbol')
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees1'], 'pool_token1_symbol')

# Add USD conversions for totalFees0
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])
fees_df_flat['count'] = 1

display(fees_df_flat.head())


fees_df_final = fees_df_flat

,timestamp,totalFees0,totalFees1,id,pool_name,pool_token0_symbol,pool_token1_symbol,index,usd_x,totalFees0_usd,usd_y,usd,totalFees1_usd,count
0,2025-09-07,0.000118,1.465512e+01,12942579003817984,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,112301.000000,13.218727,NaN,1.0,1.465512e+01,1
1,2025-09-07,0.003782,1.400000e-07,12933920349749248,Stable AMM - mUSDC/MUSD,USDC,MUSD,NaN,0.999805,0.003781,NaN,1.0,1.400000e-07,1
2,2025-09-07,0.000202,1.277600e-02,12931300419698688,Stable AMM - MUSD/mUSDT,MUSD,USDT,tether,1.000000,0.000202,1.0,NaN,NaN,1
3,2025-09-06,0.000271,2.541931e+01,12845688836587520,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,112301.000000,30.452002,NaN,1.0,2.541931e+01,1
4,2025-09-06,0.000000,8.212000e-03,12824282719584256,Stable AMM - MUSD/mUSDT,MUSD,USDT,tether,1.000000,0.000000,1.0,NaN,NaN,1


In [53]:
fees_df_final['id'].nunique(), fees_df_final['id'].count()

(217, np.int64(217))

In [48]:
total_fees_1 = fees_df_final['totalFees1_usd'].sum()
total_fees_0 = fees_df_final['totalFees0_usd'].sum()

total_fees_0, total_fees_1

(np.float64(2713.989137790969), np.float64(2875.545481801857))

In [49]:
total_fees_1 + total_fees_0

np.float64(5589.534619592826)

In [ ]:
fees_df_final['pool'] = fees_df_final['pool_name'].map(TIGRIS_MAP)

# Get Swap Fees

In [4]:
# Instantiate subgraph
swap_fees = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
swap_fee_data =  swap_fees.fetch_subgraph_data(GET_POOL_FEES, 'fees')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
Fetching transactions with skip=9000...
Fetching transactions with skip=10000...
Fetching transactions with skip=11000...
Fetching transactions with skip=12000...
Fetching transactions with skip=13000...
Fetching transactions with skip=14000...
Fetching transactions with skip=15000...
Fetching transactions with skip=16000...
Fetching transactions with skip=17000...
Fetching transactions with skip=18000...
Fetching transactions with skip=19000...
Fetching transactions with skip=20000...
Fetching transactions with skip=21000...
Fetching transactions with skip=22000...
Fetching transactions with skip=23000...
Fetching transactions with sk

In [5]:
swap_fees = pd.DataFrame(swap_fee_data)

swap_fees.head(10)

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,block_number
0,1757362749,0xd4602cd70960a06c382c2ca250e34b54841c6365,16200000000000,0,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb468e7ddd0926d8ebfa773a449d883f1564725692111...,3035515
1,1757362558,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,12480000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd02b59752593d77c51641becdf54c004ba4572a0b647...,3035462
2,1757358854,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,13047000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa527cb99de17454add617f26f5401dd717b63d7f6384...,3034434
3,1757358789,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,6000000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x90724fc436568dc0440ea441696e59ec219a20a8b621...,3034416
4,1757358778,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,7676100000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x712ce237a1f2f25e5d6b285e158900af384fa4a8c203...,3034413
5,1757358749,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,270,0,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x7ae3007829ed8a05e4b94e4fd60361a50ced75952dfe...,3034405
6,1757358698,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,116111377968,0,0x52e604c44417233b6ccedddc0d640a405caacefb,0xc5e93f157fde5eee639922404df35d68c09eb4f27241...,3034391
7,1757358659,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,10337061000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x6dad0408131d2d6d9f48cc7ae86b201f3bafafa46367...,3034380
8,1757358655,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,270952000000000,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xbaa7f54ddb616e78ac278b8e6537d6607d57e519cd44...,3034379
9,1757358572,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,160312500000,0,0x52e604c44417233b6ccedddc0d640a405caacefb,0x947a359a9c255ad10340c421085115cbd4b9d8749420...,3034356


In [7]:
sf = swap_fees.copy()

from mezo.currency_config import POOL_TOKEN_PAIRS, TOKENS_ID_MAP, POOLS_MAP
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions

sf['pool'] = sf['contractId_'].map(POOLS_MAP)

In [9]:
sf = format_datetimes(sf, ['timestamp_'])
sf.head()

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,block_number,pool
0,2025-09-08,0xd4602cd70960a06c382c2ca250e34b54841c6365,16200000000000,0,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb468e7ddd0926d8ebfa773a449d883f1564725692111...,3035515,btc_musd_pool
1,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,12480000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd02b59752593d77c51641becdf54c004ba4572a0b647...,3035462,btc_musd_pool
2,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,13047000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa527cb99de17454add617f26f5401dd717b63d7f6384...,3034434,btc_musd_pool
3,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,6000000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x90724fc436568dc0440ea441696e59ec219a20a8b621...,3034416,btc_musd_pool
4,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0,7676100000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x712ce237a1f2f25e5d6b285e158900af384fa4a8c203...,3034413,btc_musd_pool


In [10]:
sf = format_pool_token_columns(sf, 'contractId_', POOL_TOKEN_PAIRS)
sf = add_pool_usd_conversions(sf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

sf.head(10)

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,block_number,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd
0,2025-09-08,0xd4602cd70960a06c382c2ca250e34b54841c6365,1.620000e-05,0.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb468e7ddd0926d8ebfa773a449d883f1564725692111...,3035515,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,1.814773,0.000000
1,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.012480,0x52e604c44417233b6ccedddc0d640a405caacefb,0xd02b59752593d77c51641becdf54c004ba4572a0b647...,3035462,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.000000,0.012480
2,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.013047,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa527cb99de17454add617f26f5401dd717b63d7f6384...,3034434,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.000000,0.013047
3,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.006000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x90724fc436568dc0440ea441696e59ec219a20a8b621...,3034416,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.000000,0.006000
4,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.007676,0x52e604c44417233b6ccedddc0d640a405caacefb,0x712ce237a1f2f25e5d6b285e158900af384fa4a8c203...,3034413,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.000000,0.007676
5,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,2.700000e-04,0.000000,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x7ae3007829ed8a05e4b94e4fd60361a50ced75952dfe...,3034405,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999809,1.0,0.000270,0.000000
6,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,1.161114e-07,0.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xc5e93f157fde5eee639922404df35d68c09eb4f27241...,3034391,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.013007,0.000000
7,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.010337,0x52e604c44417233b6ccedddc0d640a405caacefb,0x6dad0408131d2d6d9f48cc7ae86b201f3bafafa46367...,3034380,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.000000,0.010337
8,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000000e+00,0.000271,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xbaa7f54ddb616e78ac278b8e6537d6607d57e519cd44...,3034379,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999809,1.0,0.000000,0.000271
9,2025-09-08,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,1.603125e-07,0.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x947a359a9c255ad10340c421085115cbd4b9d8749420...,3034356,btc_musd_pool,BTC,MUSD,bitcoin,NaN,112023.000000,1.0,0.017959,0.000000


In [12]:
sum(sf[['amount0_usd', 'amount1_usd']].sum())

5606.727796444897